In [20]:
import os
from dotenv import load_dotenv
from pathlib import Path

NOTEBOOK_PATH = Path().resolve()
dotenv_path = NOTEBOOK_PATH / "postgres.env"
load_dotenv(dotenv_path)

DBHOST = os.environ.get("DBHOST")
DBPORT = os.environ.get("DBPORT")
DBUSER = os.environ.get("DBUSER")
DBNAME = os.environ.get("DBNAME")
DBPASS = os.environ.get("DBPASS")

IMG_DIRECTORY = NOTEBOOK_PATH / "imgs"
IMG_DIRECTORY.mkdir(exist_ok=True)

In [21]:
from sqlalchemy import create_engine, URL

engine_url = URL.create(
    drivername="postgresql+psycopg2",
    username=DBUSER,
    password=DBPASS,
    host=DBHOST,
    port=DBPORT,
    database=DBNAME,
)

engine = create_engine(url=engine_url)

In [22]:
import pandas as pd

query = """
            SELECT 
                p.name,
                p.latitude,
                p.longitude,
                d.year, 
                d.month, 
                d.day, 
                r.amount
            FROM rainfall r
            JOIN places p
            ON p.place_id = r.place_id
            JOIN dates d
            on r.date_id = d.date_id
            ;
        """

df = pd.read_sql(query, engine)

grouped_df = df.groupby(by=["year", "month", "name", "day"]).first()

to_drop = grouped_df[grouped_df.isna().any(axis=1)].index.get_level_values(2).unique()
for name in to_drop:
    grouped_df.drop(name, level='name', inplace=True)

In [23]:
year = 2019
month = 8

idx = pd.IndexSlice
test_df = grouped_df.loc[idx[year, month],].groupby(by=["name"]).agg({"latitude": "first", "longitude": "first", "amount":"sum" })

In [24]:
import geopandas as gpd

geo_df = gpd.GeoDataFrame(
    test_df,
    geometry=gpd.points_from_xy(test_df.longitude, test_df.latitude),
    crs="EPSG:4326"
)

geo_df.reset_index(inplace=True)

In [25]:
import folium
import folium.features

interactive_map = folium.Map(
    location=(11.934, 122.969),
    zoom_start=6,
    control_scale=True,
    tiles="https://mt1.google.com/vt/lyrs=r&x={x}&y={y}&z={z}",
    attr="Google maps",
)

popup = folium.GeoJsonPopup(
    fields=["name","amount"],
    aliases=["Station Name", "Rain Amount (mm)"],
    localize=True,
    labels=True,
    style="background-color: yellow;"
)

weather_stations = folium.features.GeoJson(
    geo_df,
    name="weather station",
    popup=popup
)

weather_stations.add_to(interactive_map)

interactive_map

In [ ]:
# rainfall_layer = folium.Choropleth(
#     geo_data=intrs,
#     data=intrs,
#     columns=("id", "amount"),
#     key_on="feature.id",

#     bins=9,
#     fill_color="viridis_r",
#     line_weight=0,
#     legend_name="Rainfall Amount (mm)"
# )

# rainfall_layer.add_to(interactive_map)

# def style_function(feature):
#     return {
#         "color": "transparent",
#         "fillColor": "transparent"
#     }

# tooltip = folium.features.GeoJsonTooltip(
#     fields=("amount",),
#     aliases=("Rainfall Amount (mm)",)
# )

# tooltip_layer = folium.features.GeoJson(
#     intrs,
#     style_function=style_function,
#     tooltip=tooltip
# )
# tooltip_layer.add_to(interactive_map)

In [26]:
interactive_map.save(IMG_DIRECTORY/ f"{year}-{month}-base-map.html")